# Full Model Notebook

## OUTSTANDING WORK
* Sync train-test split
* Incorporate ALMA text preprocessing and compare performance to our preprocessing
* Ensure measurements with width greater than 5GHz are dropped (I think this is only 2)
* Band EDA
* Remove outlier projects (> 26.5 measurement) from Band prediction
* Remove projects that have incorrectly formatted band data
    * E.G. 2011.0.00008.E has an observation line with band = '3 6'
* Test different text preprocessing options and compare results
* Consider removing bands 1 AND 2 from band prediction
    * Only 21 measurements in band 1, no measurements in band 2

## Workflow Outline:
We leverage two parallel pipelines, that are combined to recommend median frequencies to explore after each model has completed training and prediction.

All projects for this phase of the overall pipeline are 'line' projects.

### Frequency Mining Pipeline
* OPTIONAL: remove projects with > 26.5 measurements **CURRENTLY REMOVING**
    * Tested both options, hit rate accuracies did not increase significantly to offset 1k cluster add

* Run projects through LDA to generate topic model with $N=50$ topics
    * Currently using count vectorization of combined title and abstract with lemmatized_no_sw_text
* Group projects to max topic by taking argmax of document-topic table
* Run HDBSCAN on each of the topics to create measurement clusters, referred to as "areas of interest"
    * Currently areas of interest are taken from min and max median frequency for each cluster generated
    * NOTE: each of the 50 HDBSCAN models can (and probably should) be tuned individually
        * We should make sure generated clusters are not too large unless it makes sense
            * E.G. a large cluster from 700-750GHz might make sense since measurements in this range are generally sparse
            * These large clusters are due to HDBSCAN adjusting the "neighborhood size", $\epsilon$ dynamically (using heirarchical clustering underneath the hood) to account for areas of varying density, as opposed to DBSCAN which uses a flat $\epsilon$ for all measurements within a topic.

### Band Prediction Pipeline
* OPTIONAL: remove projects with > 26.5 measurements **NOT CURRENTLY REMOVING NEED TO CHANGE**
* Predict band for project with Naive Bayes
    * Currently using TF-IDF vectorization of combined title and abstract with **NEED TO CHOOSE TEXT**
* Choose band(s) using hard classification into one or two bands
    * We remove band 2 entirely because there are so few 
    * We do this to be able to give a final hit rate of appx. 75%
        * This shows we have a good prediction model to match projects to band
* Ultimately we will use probability vector output (not hard classification) to order mined recommendations by full band prediction

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly_express as px
import plotly.figure_factory as ff
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import DBSCAN, HDBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

SEED = 42

In [2]:
projects = pd.read_csv("../data/nrao_projects.csv")
projects = projects.set_index('project_code')

In [3]:
line_projects = projects.query('fs_type == "line"')
line_projects.shape

(3628, 12)

In [4]:
measurements = pd.read_csv('../../nrao_measurements.csv')
measurements = measurements.set_index('project_code')
measurements = measurements[measurements.fs_type == 'line']

/var/folders/vx/9pb2fpkj3m5bkbwxzyx6sz5m0000gn/T/ipykernel_5351/229205413.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  measurements = pd.read_csv('../../nrao_measurements.csv')


## By hand band lower-bound cutoffs
To avoid possible conflicts, we simply call the cutoffs for band 1 and 2 to be 0 and 1GHz, respectively.

In [41]:
band_cutoffs = [0, 1, 84, 120, 163, 211, 275, 385, 602, 787]

## Remove outliers from projects and measurements

See 'Identifying_High_Measurement_Projects.ipynb' in 'data' folder

From this notebook, any project with > 26.5 measurements is an outlier

In [5]:
project_measurements = measurements.groupby(measurements.index)\
    .project_title.count()\
        .sort_values(ascending=False)\
        .to_frame()
project_measurements.columns = ['measurement_count']
project_measurements.head()

,measurement_count
project_code,
2017.1.00161.L,289
2017.1.00886.L,283
2021.2.00052.S,265
2023.1.00963.S,253
2022.1.00224.S,188


In [6]:
outliers = project_measurements[project_measurements.measurement_count > 26.5]

In [7]:
measurements = measurements.loc[~measurements.index.isin(outliers.index)]
line_projects = projects.loc[~projects.index.isin(outliers.index)]

### Remove measurements that have incorrectly formatted `band`

In [8]:
measurements['band'] = pd.to_numeric(measurements['band'], errors='coerce', downcast='integer')
valid_band_values = set(range(1, 11))
measurements = measurements[measurements['band'].isin(valid_band_values)] # Removing any rows with incorrect band formatting

### Make sure projects dataframe matches projects in measurement dataframes after drops

In [9]:
projects = projects.loc[measurements.index.unique()]
projects

,project_title,project_abstract,fs_type,science_category,science_keyword,band,target,raw_text,standardized_text,no_sw_text,lemmatized_sw_text,lemmatized_no_sw_text
project_code,,,,,,,,,,,,
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,Active galaxies,"Active Galactic Nuclei (AGN)/Quasars (QSO), Me...",6,1,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
2011.0.00017.S,Expanding the frontiers of chemical complexity...,The search for complex pre-biotic and biotic m...,line,ISM and star formation,"Inter-Stellar Medium (ISM)/Molecular clouds, A...",3,1,Expanding the frontiers of chemical complexity...,expanding the frontiers of chemical complexity...,expanding frontiers chemical complexity search...,expand the frontier of chemical complexity wit...,expand frontier chemical complexity search com...
2011.0.00028.S,The Effect of Extreme Environment on Protoplan...,"Protoplanetary disks, or ""proplyds"" are the si...",line,Disks and planet formation,"Disks around low-mass stars, Low-mass star for...",7,1,The Effect of Extreme Environment on Protoplan...,the effect of extreme environment on protoplan...,effect extreme environment protoplanetary disk...,the effect of extreme environment on protoplan...,effect extreme environment protoplanetary disk...
2011.0.00039.S,The ALMA view of the cool dust in an extreme l...,"We propose Band 7, extended configuration, con...",line,Active galaxies,"Starbursts, star formation, Dwarf/metal-poor g...",7,1,The ALMA view of the cool dust in an extreme l...,the view of the cool dust in an extreme low me...,view cool dust extreme low metallicity starbur...,the view of the cool dust in an extreme low me...,view cool dust extreme low metallicity starbur...
2011.0.00046.S,The first insight into the resolved molecular ...,Long-duration gamma-ray bursts (GRBs) have bee...,line,Cosmology,Gamma Ray Bursts (GRB),3,1,The first insight into the resolved molecular ...,the first insight into the resolved molecular ...,first insight resolved molecular gas propertie...,the first insight into the resolve molecular g...,first insight resolve molecular gas property h...
...,...,...,...,...,...,...,...,...,...,...,...,...
2023.1.01710.S,Mapping the host galaxies of z=2 quasars,"We propose high resolution (0.5"") observations...",line,Active galaxies,"High-z Active Galactic Nuclei (AGN), Galaxy st...",3,1,Mapping the host galaxies of z=2 quasars. We p...,mapping the host galaxies of z quasars we prop...,mapping host galaxies z quasars propose high r...,map the host galaxy of z quasar we propose hig...,map host galaxy z quasar propose high resoluti...
2023.1.01720.S,The molecular ring orbiting Sgr A*: a small pr...,There is significant observational evidence po...,line,ISM and star formation,"High-mass star formation, Pre-stellar cores, I...",6,1,The molecular ring orbiting Sgr A*: a small pr...,the molecular ring orbiting sgr a a small prom...,molecular ring orbiting sgr small promising si...,the molecular ring orbit sgr a a small promisi...,molecular ring orbit sgr small promising site ...
2023.1.01721.S,Unveiling the Early Stages of Massive Binary F...,Most of massive stars are born in binary syste...,line,ISM and star formation,High-mass star formation,7,1,Unveiling the Early Stages of Massive Binary F...,unveiling the early stages of massive binary f...,unveiling early stages massive binary formatio...,unveil the early stage of massive binary forma...,unveil early stage massive binary formation jw...


## Train-test split

In [10]:
train_texts, test_texts = train_test_split(projects.lemmatized_no_sw_text, random_state=SEED)

In [11]:
print(f'Number of train texts:{len(list(train_texts))}')
print(f'Number of test texts:{len(list(test_texts))}')

Number of train texts:2462
Number of test texts:821


In [12]:
train_texts

project_code
2022.1.01108.S    probe kinematics streamer understand star form...
2021.1.01495.S    revolutionary insight z gas dust physic althou...
2021.1.00055.S    comprehensive ism view pc scale sub l galaxy z...
2021.2.00056.S    panta rei mass energy flow parsec sub parsec s...
2018.A.00068.T    accretion burst event high mass yso g episodic...
                                        ...                        
2016.1.00615.S    probe dense gas physic extreme southern molecu...
2016.1.00744.S    investigate water deuteration young protostell...
2016.1.01344.S    multi wavelength image possibly planet induced...
2015.1.01235.S    core mass function far outer galaxy cloud dist...
2023.1.00367.S    conic cosmic noon ism condition survey cosmic ...
Name: lemmatized_no_sw_text, Length: 2462, dtype: object

### LDA class

In [13]:
class LDA_Model:
    def __init__(self, N_topics=3):
        self.N_topics = N_topics
        self.countVectorizer = CountVectorizer(stop_words='english')
        self.lda = LatentDirichletAllocation(n_components=self.N_topics, random_state=SEED)
    
    def fit(self, corpus):
        termFrequency = self.countVectorizer.fit_transform(corpus)
        self.lda.fit(termFrequency)
        return self.lda.transform(termFrequency)

    # Additional method to transform new data
    def transform(self, corpus):
        termFrequency = self.countVectorizer.transform(corpus)
        return self.lda.transform(termFrequency)

#### Initialize Model

In [14]:
lda_model = LDA_Model(N_topics=50)

#### Fit model on training set

In [15]:
train_topics = lda_model.fit(train_texts)

In [16]:
words = lda_model.countVectorizer.get_feature_names_out()

In [17]:
N = 10 #number of top words to show
topic_components = lda_model.lda.components_

for topic_idx, topic in enumerate(topic_components):
    print(f"Topic {topic_idx}:")
    # Get the indices of the top N words for this topic
    top_word_indices = topic.argsort()[-N:][::-1]
    # Print these words with their weights
    for word_idx in top_word_indices:
        print(f"{words[word_idx]} (weight: {topic[word_idx]:.2f})")
    print("\n")

Topic 0:
strip (weight: 50.65)
tail (weight: 43.06)
pressure (weight: 41.53)
gas (weight: 31.74)
ram (weight: 30.99)
molecular (weight: 23.63)
dense (weight: 23.21)
physical (weight: 14.94)
region (weight: 14.66)
nucleus (weight: 14.15)


Topic 1:
outflow (weight: 882.61)
jet (weight: 378.11)
wind (weight: 211.61)
disk (weight: 206.87)
high (weight: 196.17)
star (weight: 163.81)
molecular (weight: 158.93)
observation (weight: 152.68)
velocity (weight: 140.84)
scale (weight: 130.84)


Topic 2:
gas (weight: 351.46)
molecular (weight: 279.12)
galaxy (weight: 249.32)
cloud (weight: 147.67)
tracer (weight: 136.49)
dense (weight: 129.51)
study (weight: 114.30)
observation (weight: 107.05)
spiral (weight: 105.43)
star (weight: 101.85)


Topic 3:
molecular (weight: 91.55)
gas (weight: 61.83)
diffuse (weight: 49.20)
observation (weight: 40.88)
line (weight: 40.60)
ice (weight: 34.51)
propose (weight: 34.48)
absorption (weight: 31.04)
allow (weight: 30.79)
chemistry (weight: 28.31)


Topic 4:
di

In [18]:
train_doc_topic = pd.DataFrame(train_topics)
train_doc_topic = train_doc_topic.set_index(train_texts.index.values)
train_doc_topic.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
2022.1.01108.S,0.000235,0.000235,0.000235,0.000235,0.000235,0.000235,0.151758,0.000235,0.346698,0.000235,...,0.000235,0.000235,0.000235,0.000235,0.000235,0.000235,0.000235,0.000235,0.000235,0.000235
2021.1.01495.S,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,...,0.000171,0.000171,0.150745,0.000171,0.000171,0.219299,0.000171,0.000171,0.000171,0.000171
2021.1.00055.S,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.204172,...,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161
2021.2.00056.S,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,...,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194
2018.A.00068.T,0.000175,0.059577,0.000175,0.000175,0.000175,0.128074,0.200258,0.000175,0.000175,0.000175,...,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.145325


In [19]:
train_texts = pd.DataFrame(train_texts)

### Match test data into topics

In [20]:
test_topics = lda_model.transform(test_texts)

In [21]:
test_doc_topic= pd.DataFrame(test_topics.tolist())
test_doc_topic= test_doc_topic.set_index(test_texts.index.values)
test_doc_topic.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
2017.1.01260.S,0.000217,0.000217,0.000217,0.000217,0.000217,0.000217,0.000217,0.000217,0.000217,0.000217,...,0.000217,0.000217,0.000217,0.000217,0.139139,0.517560,0.000217,0.000217,0.000217,0.000217
2016.1.01372.S,0.000256,0.000256,0.029661,0.000256,0.000256,0.000256,0.000256,0.000256,0.000256,0.000256,...,0.000256,0.000256,0.000256,0.000256,0.000256,0.000256,0.000256,0.000256,0.000256,0.000256
2019.1.01398.S,0.000202,0.115943,0.000202,0.000202,0.000202,0.000202,0.000202,0.000202,0.000202,0.000202,...,0.000202,0.000202,0.000202,0.000202,0.063503,0.707017,0.000202,0.000202,0.000202,0.000202
2017.1.01230.S,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.137161,...,0.000222,0.000222,0.000222,0.000222,0.000222,0.348337,0.000222,0.000222,0.000222,0.000222
2021.1.00045.S,0.000171,0.000171,0.000171,0.000171,0.000171,0.058735,0.000171,0.000171,0.000171,0.000171,...,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.293903


In [22]:
test_texts = pd.DataFrame(test_texts)

### Group documents to highest matching topic

Combine project topic vector frames

In [23]:
proj_topics = pd.concat([train_doc_topic, test_doc_topic])
proj_topics

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
2022.1.01108.S,0.000235,0.000235,0.000235,0.000235,0.000235,0.000235,0.151758,0.000235,0.346698,0.000235,...,0.000235,0.000235,0.000235,0.000235,0.000235,0.000235,0.000235,0.000235,0.000235,0.000235
2021.1.01495.S,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,...,0.000171,0.000171,0.150745,0.000171,0.000171,0.219299,0.000171,0.000171,0.000171,0.000171
2021.1.00055.S,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.204172,...,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161,0.000161
2021.2.00056.S,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,...,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194
2018.A.00068.T,0.000175,0.059577,0.000175,0.000175,0.000175,0.128074,0.200258,0.000175,0.000175,0.000175,...,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.145325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019.1.01461.S,0.000182,0.000182,0.000182,0.000182,0.000182,0.000182,0.000182,0.000182,0.030220,0.000182,...,0.000182,0.000182,0.000182,0.000182,0.000182,0.077800,0.000182,0.000182,0.000182,0.000182
2023.1.01206.S,0.000183,0.000183,0.000183,0.000183,0.015246,0.000183,0.000183,0.000183,0.000183,0.268889,...,0.090452,0.000183,0.000183,0.000183,0.000183,0.302937,0.000183,0.000183,0.000183,0.000183
2021.1.00726.S,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,...,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190
2017.1.00261.S,0.000175,0.153244,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.076936,...,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.083825,0.000175


Take highest matching topic for each project

In [24]:
proj_topics['max_topic'] = proj_topics.apply(lambda x: x.argmax(), axis=1)

Create data frame with project id and max topic

In [25]:
proj_max_topic = proj_topics['max_topic'].to_frame()
proj_max_topic.head()

,max_topic
2022.1.01108.S,8
2021.1.01495.S,21
2021.1.00055.S,21
2021.2.00056.S,23
2018.A.00068.T,20


### Add `max_topic` to `measurements` frame to be able to group measurements by max topic

In [26]:
measurements = pd.merge(measurements, proj_max_topic, left_index=True, right_index=True)

In [27]:
proj_max_topic.value_counts().describe()

count     50.000000
mean      65.660000
std       87.445986
min        4.000000
25%       14.500000
50%       28.500000
75%       87.250000
max      419.000000
Name: count, dtype: float64

There are a few topics that match to a large number of documents. Perhaps we need a better topic model or to group documents by project_topic vector similarity.

Eyeball comparison of documents by max topic. This requires looking at the online explorer since printing out abstracts in here gets messy.

In [28]:
proj_max_topic[proj_max_topic.max_topic == 3].head()

,max_topic
2019.1.00799.S,3
2023.1.01573.S,3
2019.A.00023.S,3
2017.1.00575.S,3
2013.1.01194.S,3


### Generate test projects measurements
This will be useful for calculating hit rates to evaluate model performance.

**NOTE!!!**
You should not sort these, however tempting. We need to preserve the relationships of the entries to not lose measurement information.

In [29]:
test_proj_meas = measurements.loc[test_texts.index]
test_proj_meas = test_proj_meas.groupby(test_proj_meas.index)\
    .agg({
        'low_freq': lambda x: round(x, 4).tolist(),
        'high_freq': lambda x: round(x, 4).tolist(),
        'med_freq': lambda x: round(x, 4).tolist(),
        'diff_freq': lambda x: round(x, 4).tolist()
    })
test_proj_meas.head()

,low_freq,high_freq,med_freq,diff_freq
project_code,,,,
2011.0.00010.S,"[90.38, 90.7, 91.69, 92.89, 217.59, 218.67, 21...","[90.62, 90.93, 91.92, 93.12, 218.53, 219.6, 21...","[90.5, 90.815, 91.805, 93.005, 218.06, 219.135...","[0.24, 0.23, 0.23, 0.23, 0.94, 0.93, 0.94, 0.9..."
2011.0.00121.S,"[319.07, 320.48, 319.83, 319.36, 319.71, 316.59]","[320.94, 322.35, 321.71, 321.24, 321.58, 318.47]","[320.005, 321.415, 320.77, 320.3, 320.645, 317...","[1.87, 1.87, 1.88, 1.88, 1.87, 1.88]"
2011.0.00133.S,"[330.24, 332.19, 342.24, 344.24]","[332.12, 334.07, 344.12, 346.12]","[331.18, 333.13, 343.18, 345.18]","[1.88, 1.88, 1.88, 1.88]"
2011.0.00191.S,"[343.04, 344.91, 355.04, 356.91, 343.08, 344.9...","[344.91, 346.79, 356.91, 358.79, 344.96, 346.8...","[343.975, 345.85, 355.975, 357.85, 344.02, 345...","[1.87, 1.88, 1.87, 1.88, 1.88, 1.89, 1.88, 1.89]"
2011.0.00210.S,"[219.42, 219.81, 230.4, 231.18, 219.44, 219.82...","[219.66, 220.05, 230.64, 231.42, 219.67, 220.0...","[219.54, 219.93, 230.52, 231.3, 219.555, 219.9...","[0.24, 0.24, 0.24, 0.24, 0.23, 0.24, 0.24, 0.23]"


### Generate train topic measurements
We will use these to engineer 'areas of interest' among topics using DBSCAN

**NOTE!!!**
You should not sort these, however tempting. We need to preserve the relationships of the entries to not lose measurement information.

In [36]:
train_topic_freqs = measurements.loc[train_texts.index]\
    .groupby('max_topic')\
    .agg({
        'low_freq': lambda x: round(x, 4).tolist(),
        'high_freq': lambda x: round(x, 4).tolist(),
        'med_freq': lambda x: round(x, 4).tolist(),
        'diff_freq': lambda x: round(x, 4).tolist(),
        'band': lambda x: x.astype('int64').tolist()
    })
train_topic_freqs.head()

,low_freq,high_freq,med_freq,diff_freq,band
max_topic,,,,,
0,"[212.19, 214.19, 226.19, 228.19, 225.52, 239.6...","[214.06, 216.06, 228.06, 230.06, 227.52, 241.6...","[213.125, 215.125, 227.125, 229.125, 226.52, 2...","[1.87, 1.87, 1.87, 1.87, 2.0, 2.0, 1.87, 1.87,...","[6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, ..."
1,"[477.19, 478.99, 489.19, 491.14, 477.18, 489.1...","[479.19, 480.99, 491.19, 493.14, 479.18, 491.1...","[478.19, 479.99, 490.19, 492.14, 478.18, 490.1...","[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ..."
2,"[100.22, 101.72, 112.46, 114.71, 95.72, 96.97,...","[102.1, 103.6, 114.33, 115.64, 97.6, 98.84, 10...","[101.16, 102.66, 113.395, 115.175, 96.66, 97.9...","[1.88, 1.88, 1.87, 0.93, 1.88, 1.87, 1.87, 1.8...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 8, 8, 3, 3, ..."
3,"[186.31, 186.35, 186.61, 187.56, 187.86, 198.7...","[186.37, 186.4, 186.67, 187.61, 189.74, 198.78...","[186.34, 186.375, 186.64, 187.585, 188.8, 198....","[0.06, 0.05, 0.06, 0.05, 1.88, 0.06, 0.06, 0.0...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, ..."
4,"[330.56, 345.76, 330.55, 219.53, 220.37, 230.5...","[330.61, 345.82, 330.61, 219.59, 220.43, 230.5...","[330.585, 345.79, 330.58, 219.56, 220.4, 230.5...","[0.05, 0.06, 0.06, 0.06, 0.06, 0.05, 0.06, 0.1...","[7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ..."


In [ ]:
pd.DataFrame(train_topic_freqs.loc[0].med_freq,
             train_topic_freqs.loc[0].band)\
             .reset_index()

### Loop over topics and find accuracy measurements

In [39]:
test_project_hits = 0               # Hits for all projects if at least one measurement is matched
test_project_meas_hit_rate = []     # List of hit rates by project
topic_cluster_widths = []           # List of cluster widths by topic to ensure generated clusters are not too wide (list of lists)
total_num_clusters = 0              # List of number of clusters for each topic

# Loop over topics
for tpc in set(proj_max_topic.max_topic.values):
    # DBSCAN with parameters from topic parameter data frame
    # BASIC COMPARISON PARAMETERIZATION: eps=0.5, min_samples=2
    # db = DBSCAN(eps=0.25, min_samples=2)\
    #     .fit(list(zip(train_topic_freqs.loc[tpc].med_freq)))
    # db = DBSCAN(eps=params_frame.loc[tpc].eps, min_samples=2)\
    # .fit(list(zip(train_topic_freqs.loc[tpc].med_freq)))
    db = HDBSCAN(max_cluster_size=200, min_cluster_size=5)\
    .fit(list(zip(train_topic_freqs.loc[tpc].med_freq)))
    
    # Get labels from DBSCAN
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_points = len(list(labels))
    n_noise = list(labels).count(-1)

    # Stat callouts
    print(f'HDBSCAN Results for topic {tpc}')
    #print(f'Estimated number of clusters: {n_clusters}')
    print(f'Number of projects in topic: {proj_max_topic.loc[train_texts.index].query(f"max_topic == {tpc}").shape[0]}')
    print(f'Total number of measurements: {n_points}')
    print(f'Estimated number of noise measurements: {n_noise}')
    print(f'Noise percentage: {round(list(labels).count(-1)/labels.shape[0], 3)}')
    print(f'Signal to noise ratio: {round(1-list(labels).count(-1)/labels.shape[0], 3)}')

    # Create data frame for measurements in this specific topic
    selected_topic = pd.DataFrame(train_topic_freqs.loc[tpc].med_freq,
                                  train_topic_freqs.loc[tpc].band)\
    .reset_index()
    selected_topic.columns = ['band', 'med_freq']
    selected_topic['cluster_label'] = labels

    # Take mean of diff_freq and med_freq to generate areas of interest    
    topic_cluster = selected_topic.groupby('cluster_label').agg(
        mean_freq=('med_freq', 'mean'),
        min_freq=('med_freq', 'min'),
        max_freq=('med_freq', 'max'),
        count_freq=('med_freq', 'count'),
        band_min=('band', 'min'),
        band_max=('band', 'max')
    )

    topic_cluster

    # Check to see if there is noise from clustering
    # If so, drop noise cluster as we do not want to count hits there
    if (-1 in topic_cluster.index):
        topic_cluster = topic_cluster.drop(-1, axis=0) # Drop noise (label -1)
        topic_cluster.sort_values('min_freq', ascending=True)

    # Loop over generated clusters and print cluster stats
    # Initialize list of cluster widths
    # Code to check for clusters that span at least two bands
    new_rows = []
    bad_rows = []
    cluster_widths = []
    for clst in topic_cluster.index:
        min_freq, max_freq = topic_cluster.loc[clst].min_freq, topic_cluster.loc[clst].max_freq
        cluster_widths.append(max_freq - min_freq)
        total_num_clusters += 1
        if (topic_cluster.loc[clst].band_min != topic_cluster.loc[clst].band_max):
            print("BAND OVERLAP")
    print(f'Topic {tpc} cluster stats:')
    print(np.round(pd.Series(cluster_widths).describe(), 4))

    # Get a list of test project codes
    tps = proj_max_topic.loc[test_texts.index].query(f'max_topic == {tpc}')

    # Check to see if there are any test projects assigned to this topic
    # ADD IF, ELSE STATEMENT HERE, ATTACH ELSE TO FOLLOWING CODE

    #Begin test projects
    print('')
    # print('Begin tests')

    # Loop over test projects
    for tp in tps.index:
        tp_hr = 0   # Hit rate for this specific project
        #print(f'Test project {tp}:')
        # Loop over measurements in test project
        for meas in test_proj_meas.loc[tp].med_freq:
            # Loop over clusters in topic
            for clust in topic_cluster.index.values:
                lower_bound = round(topic_cluster.loc[clust].min_freq, 3)
                upper_bound = round(topic_cluster.loc[clust].max_freq, 3)
                if ((meas >= lower_bound) and (meas <= upper_bound)):
                    tp_hr += 1
                    break
        test_project_meas_hit_rate.append(round(tp_hr/len(list(test_proj_meas.loc[tp].med_freq)), 3))
        #Print some stats
        # print(f'Number of measurements: {len(test_proj_meas.loc[tp].med_freq)}')
        # print(f'Hits: {tp_hr}')
        # print(f'Hit rate: {round(tp_hr/len(list(test_proj_meas.loc[tp].med_freq)), 3)}')
        # print('')

        # Increment test_project_hits if at least one measurement in the project matched
        if (tp_hr > 0):
            test_project_hits +=1
    print('=========================================\n')

print(f'Total number of clusters across topics: {total_num_clusters}')
print(f'Number of test projects with at least one measurement match: {test_project_hits}')
print(f'Ratio of test project hits to number of test projects: {round(test_project_hits/test_texts.shape[0], 4)}')
print(f'Average hit rate per project: {round(sum(test_project_meas_hit_rate)/test_texts.shape[0], 4)}')

HDBSCAN Results for topic 0
Number of projects in topic: 5
Total number of measurements: 27
Estimated number of noise measurements: 4
Noise percentage: 0.148
Signal to noise ratio: 0.852
Topic 0 cluster stats:
count     2.0000
mean     29.2400
std       2.4183
min      27.5300
25%      28.3850
50%      29.2400
75%      30.0950
max      30.9500
dtype: float64


HDBSCAN Results for topic 1
Number of projects in topic: 128
Total number of measurements: 920
Estimated number of noise measurements: 164
Noise percentage: 0.178
Signal to noise ratio: 0.822
BAND OVERLAP
Topic 1 cluster stats:
count    76.0000
mean      1.6378
std       3.9032
min       0.0000
25%       0.1125
50%       0.4700
75%       1.2612
max      21.3300
dtype: float64


HDBSCAN Results for topic 2
Number of projects in topic: 72
Total number of measurements: 611
Estimated number of noise measurements: 100
Noise percentage: 0.164
Signal to noise ratio: 0.836
BAND OVERLAP
Topic 2 cluster stats:
count    56.0000
mean      1.